# SPECTRE PROTOTYPE
---

## Dependencies

In [5]:
import sys
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelBinarizer

from kafka import KafkaConsumer, KafkaProducer

ImportError: cannot import name 'issparse' from 'scipy.sparse' (unknown location)

## Load SavedModel

In [ ]:
model = tf.saved_model.load("/home/aryn/spectre-dev/spectre-code/spectre-ann/Model/DDOS_2/SavedModel")

## Data Preprocessing

In [ ]:
def prod_data_preprocess(df):
    def clean_dataset(df):
        assert isinstance(df, pd.DataFrame), "df needs to be a pd.DataFrame"
        df.dropna(inplace=True)
        indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)
        return df[indices_to_keep]

    def get_PCA_feature_names(num_of_pca_components):
        feature_names = []
        for i in range(num_of_pca_components):    
            feature_names.append(f"Principal component {i+1}")
        return feature_names

    # Clean the dataset.
    df = clean_dataset(df)

    # Reset the index and remove the unneeded index column.
    df = df.reset_index(drop=True)

    # Check if the 'label' column exists in the DataFrame.
    if 'label' in df.columns:
        # Save the label attribute before dropping it.
        df_labels = df['label']
        df_no_labels = df.drop('label', axis=1)
    else:
        # If the 'label' column does not exist, use the DataFrame as is.
        df_no_labels = df
        df_labels = None

    # Scale the data.
    df_scaled = StandardScaler().fit_transform(df_no_labels)

    # Perform PCA.
    pca = PCA(n_components=7)
    principal_components = pca.fit_transform(df_no_labels)

    # Create a DataFrame with principal components.
    principal_component_headings = get_PCA_feature_names(7)
    df_pc = pd.DataFrame(data=principal_components, columns=principal_component_headings)

    if df_labels is not None:
        # Concatenate principal components and labels.
        df_final = pd.concat([df_pc, df_labels], axis=1)

        # Apply LabelBinarizer to the labels.
        lb = LabelBinarizer()
        df_final['label'] = lb.fit_transform(df_final['label'])
    else:
        df_final = df_pc

    return df_final

## Kafka

## Predictions